In [1]:
import pandas as pd
import numpy as np
import datetime as dt # for date data type
import matplotlib.pyplot as plt
import numpy as np

In [2]:
df= pd.read_csv("data.csv")
df.head()

,Juhtumi nr,Toimumisaeg,Isikuid,Hukkunuid,Vigastatuid,Sõidukeid,Aadress (PPA),Maja nr (PPA),Tänav (PPA),Ristuv tänav (PPA),...,Teekatte seisund [2],Sõiduradade arv,Lubatud sõidukiirus (PPA),Tee nr (PPA),Tee km (PPA),Ilmastik [1],Valgustus [1],Valgustus [2],GPS X,GPS Y
0,2000140000057,24.10.2014 08:45,2.0,0,1,1.0,Harju maakond Tallinn Kesklinna linnaosa Esto...,9,Estonia pst,NaN,...,Kuiv,NaN,50.0,NaN,NaN,Selged olud,Valge aeg,Valge aeg,6588678.0,542647.0
1,2000140000067,24.10.2014 13:45,2.0,0,1,2.0,Harju maakond Tallinn Põhja-Tallinna linnaosa...,NaN,Kopli tn,Telliskivi tn,...,Kuiv,NaN,30.0,NaN,NaN,Pilvised olud,Valge aeg,Valge aeg,6589522.0,541467.0
2,2000140000123,8/11/14,2.0,0,1,2.0,Harju maakond Tallinn Pirita linnaosa Meriväl...,NaN,Merivälja tee,Mähe tee,...,Kuiv,NaN,50.0,NaN,NaN,Pilvised olud,Valge aeg,Valge aeg,6593961.0,547646.0
3,2000140000235,17.11.2014 17:32,2.0,0,2,2.0,Harju maakond Keila vald,NaN,KEILA - HAAPSALU,NaN,...,Kuiv,NaN,90.0,17.0,"10,95",Pilvised olud,Pimeda aeg,Pimeda ajal valgustus ei põle,6569324.0,516628.0
4,2000150000442,28.04.2015 07:55,2.0,0,1,2.0,Harju maakond Tallinn Kesklinna linnaosa Pärn...,123a,Pärnu mnt,NaN,...,Märg,NaN,50.0,NaN,NaN,Vihmasadu,Valge aeg,Valge aeg,6586430.0,541953.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15708 entries, 0 to 15707
Data columns (total 53 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   Juhtumi nr                                     15708 non-null  object 
 1   Toimumisaeg                                    15708 non-null  object 
 2   Isikuid                                        15633 non-null  float64
 3   Hukkunuid                                      15708 non-null  int64  
 4   Vigastatuid                                    15708 non-null  int64  
 5   Sõidukeid                                      15633 non-null  float64
 6   Aadress (PPA)                                  14601 non-null  object 
 7   Maja nr (PPA)                                  6686 non-null   object 
 8   Tänav (PPA)                                    14012 non-null  object 
 9   Ristuv tänav (PPA)                             275

In [4]:
from shapely.geometry import Point
import pyproj


# Convert data types to date/time, int etc
veerud_int=['Isikuid', 'Hukkunuid', "Vigastatuid", "Sõidukeid", "Kergliikurijuhi osalusel", "Jalakäija osalusel", "Kaassõitja osalusel", "Maastikusõiduki juhi osalusel", "Eaka (65+) mootorsõidukijuhi osalusel", "Bussijuhi osalusel", "Veoautojuhi osalusel", "Ühissõidukijuhi osalusel", "Sõiduautojuhi osalusel", "Mootorratturi osalusel", "Mopeedijuhi osalusel", "Jalgratturi osalusel", "Alaealise osalusel", "Turvavarustust mitte kasutanud isiku osalusel", "Esmase juhiloa omaniku osalusel", "Mootorsõidukijuhi osalusel", "Lubatud sõidukiirus (PPA)", "Tee nr (PPA)",  "GPS X", "GPS Y"]
veerud_cat=["Maakond (PPA)", "Omavalitsus (PPA)", "Asustus (PPA)", "Liiklusõnnetuse liik [1]", "Liiklusõnnetuse liik [3]", "Tüüpskeemi nr",  "Tüüpskeem [2]", "Tee tüüp [1]", "Tee tüüp [2]", "Tee element [1]", "Tee element [2]", "Tee objekt [2]", "Kurvilisus", "Tee tasasus", "Tee seisund", "Teekate", "Teekatte seisund [2]", "Sõiduradade arv", "Ilmastik [1]", "Valgustus [1]", "Valgustus [2]"]

df["Toimumisaeg2"]=pd.to_datetime(df["Toimumisaeg"], format='mixed')
df[veerud_int] = df[veerud_int].astype('Int64', errors='ignore')
df[veerud_cat]=df[veerud_cat].astype("category", errors='ignore')



# Drop rows without GPS coordinates
df.dropna(subset=['GPS Y', 'GPS Y'], inplace=True)

# Drop rows without Time
df.dropna(subset=['Toimumisaeg2'], inplace=True)

df['Asukoht'] = df.apply(lambda row: Point(float(row['GPS Y']), float(row['GPS X']))
                          if not pd.isna(row['GPS X']) and not pd.isna(row['GPS Y']) else None,
                          axis=1)

# Define the source and target CRS
source_crs = pyproj.CRS.from_epsg(3301)  # EPSG 3301
target_crs = pyproj.CRS.from_epsg(4326)  # EPSG 4326

# Create a transformer to convert coordinates from source CRS to target CRS
transformer = pyproj.Transformer.from_crs(source_crs, target_crs, always_xy=True)

# Transform the Point coordinates
df['Asukoht'] = df['Asukoht'].apply(lambda point: Point(transformer.transform(point.x, point.y)) if point is not None else None)

# outside Estonia - possible outliers
# Assuming the bounding box coordinates are defined as follows:
#min_lat = 57.5097   # Minimum latitude (southernmost point of Estonia)
#max_lat = 59.6750   # Maximum latitude (northernmost point of Estonia)
#min_lon = 21.3826   # Minimum longitude (westernmost point of Estonia)
#max_lon = 28.2090   # Maximum longitude (easternmost point of Estonia)

# Filter out GPS points outside the bounding box
#filtered_df = df[(df["GPS X"] >= min_lon) & (df["GPS X"] <= max_lon) & (df["GPS Y"] >= min_lat) & (df["GPS Y"] <= max_lat)]
df



,Juhtumi nr,Toimumisaeg,Isikuid,Hukkunuid,Vigastatuid,Sõidukeid,Aadress (PPA),Maja nr (PPA),Tänav (PPA),Ristuv tänav (PPA),...,Lubatud sõidukiirus (PPA),Tee nr (PPA),Tee km (PPA),Ilmastik [1],Valgustus [1],Valgustus [2],GPS X,GPS Y,Toimumisaeg2,Asukoht
0,2000140000057,24.10.2014 08:45,2,0,1,1,Harju maakond Tallinn Kesklinna linnaosa Esto...,9,Estonia pst,NaN,...,50,<NA>,NaN,Selged olud,Valge aeg,Valge aeg,6588678,542647,2014-10-24 08:45:00,POINT (24.751478719314402 59.43379595290958)
1,2000140000067,24.10.2014 13:45,2,0,1,2,Harju maakond Tallinn Põhja-Tallinna linnaosa...,NaN,Kopli tn,Telliskivi tn,...,30,<NA>,NaN,Pilvised olud,Valge aeg,Valge aeg,6589522,541467,2014-10-24 13:45:00,POINT (24.730849754494095 59.44148846858757)
2,2000140000123,8/11/14,2,0,1,2,Harju maakond Tallinn Pirita linnaosa Meriväl...,NaN,Merivälja tee,Mähe tee,...,50,<NA>,NaN,Pilvised olud,Valge aeg,Valge aeg,6593961,547646,2014-08-11 00:00:00,POINT (24.840723605498106 59.480681179884726)
3,2000140000235,17.11.2014 17:32,2,0,2,2,Harju maakond Keila vald,NaN,KEILA - HAAPSALU,NaN,...,90,17,"10,95",Pilvised olud,Pimeda aeg,Pimeda ajal valgustus ei põle,6569324,516628,2014-11-17 17:32:00,POINT (24.29152856115738 59.26188409298864)
4,2000150000442,28.04.2015 07:55,2,0,1,2,Harju maakond Tallinn Kesklinna linnaosa Pärn...,123a,Pärnu mnt,NaN,...,50,<NA>,NaN,Vihmasadu,Valge aeg,Valge aeg,6586430,541953,2015-04-28 07:55:00,POINT (24.738814527574075 59.41368770550173)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15702,3140170029400,20.05.2017 23:29,1,0,1,2,Saare maakond Lääne-Saare vald,NaN,KURESSAARE RINGTEE,NaN,...,90,76,11,Selged olud,Pimeda aeg,Pimeda ajal valgustus puudub,6457753,408283,2017-05-20 23:29:00,POINT (22.437819090257758 58.25094911374109)
15703,3140170029504,21.05.2017 09:35,3,0,1,2,Pärnu maakond VANA Audru vald Jõõpre küla,NaN,AUDRU - LAVASSAARE - VAHENURME,NaN,...,90,19103,7,Selged olud,Valge aeg,Valge aeg,6480481,519847,2017-05-21 09:35:00,POINT (24.34006762403213 58.46414855123448)
15704,3140170029565,21.05.2017 13:50,2,0,1,3,Rapla maakond Märjamaa vald Konuvere küla,NaN,NaN,NaN,...,90,<NA>,NaN,Madal vastu paistev päike,Valge aeg,Valge aeg,6519056,524082,2017-05-21 13:50:00,POINT (24.416736737406524 58.810260868978204)
15705,3140170029903,22.05.2017 18:29,1,0,1,1,Harju maakond Nissi vald Jaanika küla,NaN,ÄÄSMÄE - HAAPSALU - ROHUKÜLA,NaN,...,90,9,14,Selged olud,Valge aeg,Valge aeg,6558038,519643,2017-05-22 18:29:00,POINT (24.34337494890923 59.16044314701935)


In [5]:
import folium
from folium.plugins import MarkerCluster

# Create a map centered on Estonia
map_osm = folium.Map(location=[59.4370, 24.7536], zoom_start=10)

# Create a MarkerCluster layer
marker_cluster = MarkerCluster().add_to(map_osm)

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    asukoht = row['Asukoht']
    
    # Check if the 'Asukoht' value is not None
    if asukoht is not None:
        latitude = asukoht.y
        longitude = asukoht.x
        
        # Create a marker for each point and add it to the MarkerCluster layer
        folium.Marker(location=[latitude, longitude]).add_to(marker_cluster)

# Display the map
map_osm


In [6]:
df['lon'] = df.Asukoht.apply(lambda p: p.x)
df['lat'] = df.Asukoht.apply(lambda p: p.y)

In [7]:
#export data
df.to_csv('df_clean.csv')
